In [1]:
from bot import IELTSBot
import numpy as np
import torch

2024-05-06 00:46:42.745151: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 00:46:42.745176: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 00:46:42.745774: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 00:46:43.279263: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
EXAMPLES = 'sample/cambridge'

In [3]:
import os.path
dirs = [d for d in os.listdir(EXAMPLES) if os.path.isdir(os.path.join(EXAMPLES, d))]
print(dirs)

['15_2_6', '17_3_6.5', '15_1_7', '16_3_7', '17_1_6.5', '16_4_4', '16_2_4.5', '17_2_6.5', '15_4_6.5', '15_3_7', '17_4_6', '16_1_6']


In [4]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [5]:
scoring_bot = IELTSBot(model_name=model_id, verbose=True, quantization='int4')

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Evaluating results

In [6]:
# scoring_bot.change_system_prompt_criteria(False)

In [7]:
# print(scoring_bot.system_prompt)


    You are an English teaching assistant, and you are good at evaluating essays, and your students need you for their IELTS Academic essay task 2. You will make evaluation based on the topic and student's response.
    You should grade the essay general score and in 4 metrics in IELTS Writing, which are Task Response, Coherence and Cohesion, Lexical Resource and Grammatical Range and Accuracy. 
    The overall must be the mean value of 4 metric scores and can be a float value between 0 and 9 (round to .5), but each metric score should be an integer between 0 and 9.
    Recall the IELTS Writing band score criteria.
    
    The formula to calculate the general score is:
    ```
    General_score = ( ( Task_Response + Coherence_and_Cohesion + Lexical_Resource + Grammatical_Range_and_Accuracy) // 2 ) / 2 
    ```

    In each metric, you should give a detailed explanation and point out exactly the student mistakes that led to that score.
    Your output format should be like this:
    *

In [6]:
y_true = []
y_preds = []

In [7]:
scoring_bot.clear_messages()

In [8]:
with open(f'{EXAMPLES}/15_1_7/question.txt', 'r') as f:
                essay_topic = f.read()

with open(f'{EXAMPLES}/15_1_7/answer.txt', 'r') as f:
    student_response = f.read()

In [9]:
scoring_bot.messages

[{'role': 'system',
  'content': "\nYou are an English teaching assistant, and you are good at evaluating essays, and your students need you for their IELTS Academic essay task 2. You will make evaluation based on the topic and student's response.\nYou should grade the essay general score and in 4 metrics in IELTS Writing, which are Task Response, Coherence and Cohesion, Lexical Resource and Grammatical Range and Accuracy. \nThe overall must be the mean value of 4 metric scores and can be a float value between 0 and 9 (round to .5), but each metric score should be an integer between 0 and 9.\nRecall the IELTS Writing band score criteria.\nHere are the criteria for each metric at each band score:\n\n**Task Response:**\nBand 8:\nThe prompt is appropriately and sufficiently addressed.\nA clear and well-developed position is presented in response to the question.\nIdeas are relevant, well extended and supported.\nThere may be occasional omissions or lapses in content.\n\nBand 7:\nThe main 

In [10]:
for is_rag in [False]:
    if is_rag:
        scoring_bot.setup_rag()
    else:
        scoring_bot.remove_rag()
    for criteria in [False, True]:
        scoring_bot.change_system_prompt_criteria(criteria)
        for is_rescore in [True ,False]:
            y = []
            y_pred = []
            
            for dir in dirs:
                with open(f'{EXAMPLES}/{dir}/question.txt', 'r') as f:
                    essay_topic = f.read()

                with open(f'{EXAMPLES}/{dir}/answer.txt', 'r') as f:
                    student_response = f.read()
                    
                score = float(dir.split('_')[-1])
                y.append(score)
                scoring_bot.generate_response(essay_topic, student_response, is_rescore=is_rescore)
                y_pred.append(scoring_bot.general)
                
                scoring_bot.clear_messages()
                
            y = np.array(y)
            y_pred = np.array(y_pred)
            
            y_true.append(y)
            y_preds.append(y_pred)
            
            print(f'is_rag: {is_rag} is_rescore: {is_rescore}, criteria: {criteria}, MSE: {np.mean((y - y_pred)**2)}')


/home/quanghung20gg/anaconda3/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 2_______
Inconsistent scoring


--- Logging error ---
Traceback (most recent call last):
  File "/home/quanghung20gg/anaconda3/lib/python3.11/logging/__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "/home/quanghung20gg/anaconda3/lib/python3.11/logging/__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "/home/quanghung20gg/anaconda3/lib/python3.11/logging/__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "/home/quanghung20gg/anaconda3/lib/python3.11/logging/__init__.py", line 377, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/quanghung20gg/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launc

______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 2_______
Inconsistent scoring
______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 2_______
Inconsistent scoring
______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 2_______
Inconsistent scoring
______Adjusting score 1_______
Inconsistent scoring
______Adjusting score 2_______
Inconsistent scoring
______Adjusting score 3_______
______Adjusting score 4_______
Inconsistent scoring
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/quanghung20gg/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_74186/787516691.py", line 21, in <module>
    scoring_bot.generate_response(essay_topic, student_response, is_rescore=is_rescore)
  File "/home/quanghung20gg/Documents/hung20gg/ielts_scoring/bot.py", line 345, in generate_response
    if not self.is_rag:
        ^^^^^^^^^^^^^^^
  File "/home/quanghung20gg/Documents/hung20gg/ielts_scoring/bot.py", line 209, in _rescore
  File "/home/quanghung20gg/anaconda3/lib/python3.11/site-packages/transformers/pipelines/text_generation.py", line 235, in __call__
    return super().__call__(Chat(text_inputs), **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/quanghung20gg/anaconda3/lib/python3.11/site-packages/transformers/pipelines/base.py", line 1206, in __call__
    return self.run_si

In [16]:
y, y_pred

([6.0, 6.5, 7.0, 7.0, 6.5, 4.0, 4.5, 6.5, 6.5, 7.0],
 [7.5, 6.5, 4.5, 7.0, 6.0, 5.0, 5.0, 6.5, 7.0])

In [21]:
print(y_preds[6])
yy = (((y_preds[4] + y_preds[6])*2)//2)/2
np.mean((y_true[4] - yy)**2)

[8.  6.5 7.  6.  7.  4.  5.  6.  6.5 6.5 6.5 6. ]


0.4791666666666667

In [6]:
response = scoring_bot.generate_response(essay_topic, student_response, is_rescore=True)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


______Adjusting score 1_______
Inconsistent scoring


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


______Adjusting score 1_______
Inconsistent scoring


In [7]:
scoring_bot.get_message()

[{'role': 'system',
  'content': "\n    You are an English teaching assistant, and you are good at grading essays, and your students need you for their IELTS Academic essay task 2. You will be given the topic and student's response.\n    You should grade the essay general score and in 4 metrics in IELTS Writing, which are `Task Response`, `Coherence and Cohesion`, `Lexical Resource` and `Grammatical Range and Accuracy`. \n    The overall score can be a float between 0 and 9 (round to .5), but each metric score should be an integer between 0 and 9.\n\n    Recall the IELTS Writing band score criteria\n\n    Here are the criteria for each metric at each band score:\n    Recall the criteria for each metric in common band score:\n\n**Task Response:**\nBand 8:\nThe prompt is appropriately and sufficiently addressed.\nA clear and well-developed position is presented in response to the question.\nIdeas are relevant, well extended and supported.\nThere may be occasional omissions or lapses in c

In [8]:
print(response)

**General: 6.0**
Overall Explanation: The essay attempts to discuss the reasons behind people's interest in knowing the history of their place of residence, and while it has some limitations, it still conveys the writer's ideas reasonably well.

**Task Response: 7**
Explanation: The essay partially addresses the topic, but it does so in a clear and concise manner. The writer identifies two main reasons for people's interest in knowing the history of their place of residence, and provides some supporting evidence. The essay lacks a clear thesis statement, but the writer's ideas are generally relevant to the topic.

**Coherence and Cohesion: 6**
Explanation: The essay has some coherence, with the writer's ideas flowing reasonably well from one paragraph to the next. However, the transitions between paragraphs could be improved, and the writer could use more cohesive devices to connect their ideas.

**Lexical Resource: 6**
Explanation: The writer uses a limited range of vocabulary, but th

In [9]:
indepth_feedback = scoring_bot.indepth_feedback()
print(indepth_feedback)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**General: 6.0**
Overall Explanation: The essay attempts to discuss the reasons behind people's interest in knowing the history of their place of residence, and while it has some limitations, it still conveys the writer's ideas reasonably well. However, the essay lacks clarity, coherence, and accuracy in some areas, which affects its overall score.

**Task Response: 7**
Explanation: The essay partially addresses the topic, but it does so in a clear and concise manner. The writer identifies two main reasons for people's interest in knowing the history of their place of residence, and provides some supporting evidence. However, the essay lacks a clear thesis statement, and the writer does not fully explore the reasons behind this phenomenon. For example, the writer mentions "human nature is full of curiosity" as a reason, but does not elaborate on this point. Additionally, the writer does not fully discuss the second reason mentioned, "firmness of building is another strong rational". Th

In [10]:
revived_essay = scoring_bot.revive_essay()
print(revived_essay)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '**Suggesting changes:**\n\n**Task Response:**\n\n* Add a clear thesis statement to introduce the main argument: "As people become increasingly interested in knowing the history of their place of residence, it is essential to understand the reasons behind this phenomenon."\n* Provide more depth and support for the two reasons mentioned: "Firstly, human nature is full of curiosity, and people are naturally inclined to explore the past of their living space. Secondly, the concern for the soundness of the building structure and the potential issues it may pose are significant factors driving people\'s interest in knowing the history of their place of residence."\n\n**Coherence and Cohesion:**\n\n* Add a clear topic sentence to each paragraph to provide a clear direction for the reader: "The first reason people are interested in knowing the history of their place of residence is due to human nature\'s curiosity."\n* Use transitional phrases to connect the ideas between 

In [11]:
print(scoring_bot.get_revived_essay())



In many nations, people are developing a growing interest in the historical background of their living space. The current essay discusses the reasons behind this phenomenon.

There are two main reasons for this phenomenon. Firstly, human nature is full of curiosity, and this inclination to explore the past is a driving force behind people's interest in knowing the history of their place of residence. Secondly, the concern for the soundness of the building structure and the potential issues it may pose are significant factors driving people's interest in knowing the history of their place of residence. People also wonder about who has lived in this building before them and what were their experiences related to the place.

There are many methods to discover these curiosities. Firstly, people can check the history of the building in public records. In this way, they are able to know who built the building and all other confusions related to the potential soundness of the building. Seco

In [12]:
scoring_bot.clear_messages()